In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import ExponentialSmoothing

def triple_additive_smoothing_sm(data: pd.Series, slen: int, future_periods: int):
    """
    Performs Triple Additive Exponential Smoothing (Holt-Winters' Additive Method) using the statsmodels library.

    Args:
        data (pd.Series): The time series of historical demand. Must have a DatetimeIndex.
        slen (int): The length of the seasonal period (e.g., 12 for monthly data with
                     a yearly season, 7 for daily data with a weekly season).
        future_periods (int): The number of periods to forecast into the future.

    Returns:
        tuple[pd.DataFrame, any]: 
            - A DataFrame containing the original data, the forecast, and the error.
            - The fitted statsmodels results object for further analysis.
    """
    # --- 1. Input Validation ---
    # A robust model needs at least two full seasons of data to learn the pattern.
    if len(data) < 2 * slen:
        raise ValueError(f"Data must have at least 2 full seasons of data for this model. "
                         f"Need {2 * slen} data points, but got {len(data)}.")

    # --- 2. Initialize and Fit the Model ---
    # We initialize the ExponentialSmoothing model with our data and specify the model type.
    # 'add' stands for additive.
    model = ExponentialSmoothing(
        data,
        trend='add',                # Additive trend
        seasonal='add',             # Additive seasonality
        seasonal_periods=slen,      # Length of the season
        initialization_method='estimated' # Let statsmodels estimate initial values
    )

    # Fit the model. Statsmodels will automatically find the best alpha, beta, and gamma.
    fitted_model = model.fit(optimized=True)

    # --- 3. Generate Forecasts ---
    # Get the historical forecast (in-sample predictions)
    historical_forecast = fitted_model.fittedvalues

    # Forecast future periods (out-of-sample predictions)
    future_forecast = fitted_model.forecast(steps=future_periods)

    # --- 4. Consolidate Results into a DataFrame ---
    # Combine the historical and future forecasts into a single series
    full_forecast = pd.concat([historical_forecast, future_forecast])

    # Create the final DataFrame for analysis
    results_df = pd.DataFrame({
        'Demand': data,
        'Forecast': full_forecast
    })
    
    # Calculate the forecast error
    results_df['Error'] = results_df['Forecast'] - results_df['Demand']
    
    print("Successfully fitted Holt-Winters' Additive model.")
    
    return results_df, fitted_model